<a href="https://colab.research.google.com/github/OnurBasci/ARGENIT_Internship/blob/main/Chromosome_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

first deep learning project

imports


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision
import torchvision.transforms as transforms


drivea erisim

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


unzip the dataset

In [ ]:
!cp -r /content/gdrive/MyDrive/staj/chromosome_segmentation/clasify_dataset /content/datasets

In [ ]:
%cd /content/datasets/clasify_dataset
!unzip /content/datasets/cuted_image.zip

Class for custom dataset

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from skimage import io

In [ ]:
class chromosomes(Dataset):
  def __init__(self, csv_file, root_dir, transform=None):
    self.annotations = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform
  def __len__(self):
    return len(self.annotations)
  
  def __getitem__(self, index):
    img_path = os.path.join(self.root_dir, str(pd.read_csv("/content/datasets/z_labels.csv").iloc[index, 0]).split(";")[0]) #the most stupid code ever
    image = io.imread(img_path)
    y_label = torch.tensor(int(pd.read_csv("/content/datasets/z_labels.csv").iloc[index, 0].split(";")[1]))

    if self.transform:
      image = self.transform(image)
    
    return (image, y_label)

In [ ]:
print(str(pd.read_csv("/content/datasets/z_labels.csv").iloc[71, 0]).split(";")[1])
#print(str(pd.read_csv("z_labels.csv").iloc[0]).split(";")[1])

23


custom datasetler icin

In [ ]:
batch_size = 64
dataset = chromosomes(csv_file = '/content/datasets/z_labels.csv', root_dir = "/content/datasets/clasify_dataset/cuted_image", transform = transforms.ToTensor())
train_set, test_set = torch.utils.data.random_split(dataset, [20000, 3000])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

Parameters

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = 2500 #50*50 size of images
num_classes = 24
learning_rate = 0.01
num_epochs = 24
in_channel = 3

googlenet model

In [ ]:
model = torchvision.models.googlenet(pretrained=True)
model.to(device)

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


  0%|          | 0.00/49.7M [00:00<?, ?B/s]

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

Resnet model

In [ ]:
model = torchvision.models.resnet18()
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Mnist dataseti icin

In [ ]:
batch_size = 64
train_dataset = datasets.MNIST(
    root="dataset/",
    train=True,
    transform=transforms.ToTensor(),
    download = True
)
train_loader = DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True
)
test_dataset = datasets.MNIST(
    root="dataset/",
    train = False,
    transform = transforms.ToTensor(),
    download=True
)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=True
)


define the neural network


In [ ]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x


In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

In [ ]:
for epoch in range(num_epochs):
  print(f"Epoch: {epoch}")
  for batch_idx, (data, targets) in enumerate(train_loader):
    print(batch_idx)
    #get data to cuda if possible
    data = data.to(device=device)
    targets = targets.to(device=device)

    #get to correct shape 28*28-> 784
    #data = data.reshape(data.shape[0], -1)

    #forward propagation
    scores = model(data)
    loss = criterion(scores, targets)

    #zero previous gradients
    optimizer.zero_grad()

    #back-propagation
    loss.backward()

    #Adam descent
    optimizer.step()

Epoch: 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
2

In [ ]:
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0

  #put he model in the evaluation mode
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x= x.to(device=device)
      y = y.to(device=device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

      print(
          f"Got {num_correct}/{num_samples} with accuracy"
          f"{float(num_correct) / float(num_samples) * 100:.2f}"
      )

      model.train()

In [ ]:
#check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Got 0/64 with accuracy0.00
Got 0/128 with accuracy0.00
Got 0/192 with accuracy0.00
Got 0/256 with accuracy0.00
Got 0/320 with accuracy0.00
Got 0/384 with accuracy0.00
Got 0/448 with accuracy0.00
Got 0/512 with accuracy0.00
Got 0/576 with accuracy0.00
Got 0/640 with accuracy0.00
Got 0/704 with accuracy0.00
Got 0/768 with accuracy0.00
Got 0/832 with accuracy0.00
Got 0/896 with accuracy0.00
Got 0/960 with accuracy0.00
Got 0/1024 with accuracy0.00
Got 0/1088 with accuracy0.00
Got 0/1152 with accuracy0.00
Got 0/1216 with accuracy0.00
Got 0/1280 with accuracy0.00
Got 0/1344 with accuracy0.00
Got 0/1408 with accuracy0.00
Got 0/1472 with accuracy0.00
Got 0/1536 with accuracy0.00
Got 0/1600 with accuracy0.00
Got 0/1664 with accuracy0.00
Got 0/1728 with accuracy0.00
Got 0/1792 with accuracy0.00
Got 0/1856 with accuracy0.00
Got 0/1920 with accuracy0.00
Got 0/1984 with accuracy0.00
Got 0/2048 with accuracy0.00
Got 0/2112 with accuracy0.00
Got 0/2176 with accuracy0.00
Got 0/2240 with accuracy0.00
G